# Kinesis Streaming Consumer

In [ ]:
from boto import kinesis
import datetime
import time
import dateutil.parser
STREAM_NAME="latency"

In [2]:
kinesis_conn = kinesis.connect_to_region("us-east-1")
streams = kinesis_conn.describe_stream(STREAM_NAME)
streams

{u'StreamDescription': {u'EnhancedMonitoring': [{u'ShardLevelMetrics': []}],
  u'HasMoreShards': False,
  u'RetentionPeriodHours': 24,
  u'Shards': [{u'HashKeyRange': {u'EndingHashKey': u'340282366920938463463374607431768211455',
     u'StartingHashKey': u'0'},
    u'SequenceNumberRange': {u'StartingSequenceNumber': u'49569120833560767652834857066126646824474736685823819778'},
    u'ShardId': u'shardId-000000000000'}],
  u'StreamARN': u'arn:aws:kinesis:us-east-1:157926643553:stream/latency',
  u'StreamCreationTimestamp': 1483312335.0,
  u'StreamName': u'latency',
  u'StreamStatus': u'ACTIVE'}}

In [3]:
run_timestamp=datetime.datetime.now()
RESULT_FILE= "results/kinesis-latency-" + run_timestamp.strftime("%Y%m%d-%H%M%S") + ".csv"
try:
    os.makedirs("results")
except:
    pass
output_file=open(RESULT_FILE, "w")

In [4]:
shard_ids = [s["ShardId"] for s in streams['StreamDescription']["Shards"]]
print str(shard_ids)

shards ={}
for sid in shard_ids:
    shards[sid]=kinesis_conn.get_shard_iterator(STREAM_NAME, sid, "LATEST")["ShardIterator"] 


while True:
    for sid in shards.keys():
        shard_it=shards[sid]
        #print "Get Record Shard ID: %s It: %s " %(sid, str(shard_it))
        record = kinesis_conn.get_records(shard_it, limit=10000)
        #print record
        records_set = record["Records"]
        number_records = len(records_set)
        #print "Received %d records"%number_records
        #print "Process Record It: " + str(shard_it)
        shards[sid] = record["NextShardIterator"]                   
        if len(records_set)>0:            
            for r in records_set:                
                message = r["Data"]
                #print message
                now = time.time()
                #sent_time=datetime.datetime.strptime(message.value, "%Y-%m-%dT%H:%M:%S.%fZ")
                sent_time_string = message.split(";")[0]
                sleep_time =float(message.split(";")[1])
                sent_time = dateutil.parser.parse(sent_time_string)
                sent_time_ts = time.mktime(sent_time.timetuple())
                lat = now-sent_time_ts    
                result = "kinesis, latency, 0, %.5f, %.5f\n"%(1/sleep_time, lat)
                print(result)
                output_file.write(result)
                output_file.flush()
        time.sleep(0.21)

[u'shardId-000000000000']
kinesis, latency, 1.00000, 0.52159

kinesis, latency, 1.00000, 0.62351

kinesis, latency, 1.00000, 0.72740

kinesis, latency, 1.00000, 0.82874

kinesis, latency, 1.00000, 0.93329

kinesis, latency, 1.00000, 1.03626

kinesis, latency, 1.00000, 0.87298

kinesis, latency, 1.00000, 0.98438

kinesis, latency, 1.00000, 1.09891

kinesis, latency, 1.00000, 1.20871

kinesis, latency, 1.00000, 1.31142

kinesis, latency, 1.00000, 0.14126

kinesis, latency, 1.00000, 0.25027

kinesis, latency, 1.00000, 0.35524

kinesis, latency, 1.00000, 0.36185

kinesis, latency, 1.00000, 0.47539

kinesis, latency, 1.00000, 0.59409

kinesis, latency, 1.00000, 0.70171

kinesis, latency, 1.00000, 0.53247

kinesis, latency, 1.00000, 0.64374

kinesis, latency, 10.00000, 0.68712

kinesis, latency, 10.00000, 0.96546

kinesis, latency, 10.00000, 0.96578

kinesis, latency, 10.00000, 0.26948

kinesis, latency, 10.00000, 0.54896

kinesis, latency, 10.00000, 0.54928

kinesis, latency, 10.00000, 0.82

ResourceNotFoundException: ResourceNotFoundException: 400 Bad Request
{u'message': u'Shard shardId-000000000000 in stream latency under account 157926643553 does not exist', u'__type': u'ResourceNotFoundException'}

In [ ]:
!ls results